## Protein Classification

The goal of this notebook is to reproduce the results from:
- Cang, Z., Mu, L., Wu, K., et al. (2015). **A topological approach for protein classification.** Computational and Mathematical Biophysics, 3(1)
- Xia, K. & Wei, G. W. (2014). **Persistent homology analysis of protein structure, flexibility, and folding**. International journal for numerical methods in biomedical engineering, 30(8), 814–844
- Pun, C. S., Xia K., Lee, S. X. (2018) **Persistent-Homology-based Machine Learning and its Applications -- A Survey.** arXiv:1811.00252 


## 1. The problem

The problem consist of classify the secondary structure of proteins from its embedding in $\mathbb{R}^3$.


<img src="../figures/proteins.png" width="700">

We use the data available at the protein database **SCOPe**.
- N. K. Fox, S. E. Brenner, and J.M. Chandonia. (2014)  Scope:  **Structural classification of proteins-extended,  integrating scop and astral data and classification of new structures.** Nucleic acids research, 42(D1):D304–D309.

See examples: 
    
- **all alpha**: https://scop.berkeley.edu/sunid=16311
- **all beta**: https://scop.berkeley.edu/sunid=22836
- **alpha and beta:** https://scop.berkeley.edu/sunid=28513

The main idea is to quantify geometric properties of the structure of proteins from its persistence diagrams.

<img src="../figures/alpha.jpg" width="600">

<img src="../figures/beta.jpg" width="600">

In [4]:
#!pip install biopython
from Bio.PDB import *
import numpy as np
import urllib.request

#!pip install ripser
from ripser import Rips
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

We will next download from the data bank SCOPe, the files .pdb from proteins we will study. 

Create _all_alphas folder_ in your _data_ directory. Then, download the structure of each of the proteins described at  the file *all_alphas.txt* in the folder _data_.


In [5]:
all_alphas = np.loadtxt('../data/all_alphas.txt', dtype='str').ravel()
for x in all_alphas:  
    urllib.request.urlretrieve("https://scop.berkeley.edu/astral/pdbstyle/ver=2.07&id="+ x + "&output=html", "../data/all_alphas/"+ x +".pdb")

Repeat the previous procedure with the classes _all_betas_ and _alpha_beta_.

In [ ]:
#Code to download the data of proteins all_betas

In [ ]:
#Code to download the data of proteins alpha_beta

In [1]:
#The following function computes the intrinsic distance between carbon atoms in the protein.
def get_distance_matrix(structure):
    if len(structure.get_list())>0:
        model = structure.get_list()[0]
        chain = model.get_list()[0]
        N = len(chain)
        i = 0
        j = 0
        distance=np.empty([N,N])
        for residue1 in chain:
            for residue2 in chain:
                # compute distance between CA atoms
                try:
                    distance[i%N,j%N] = residue1['CA'] - residue2['CA']
                except KeyError:
                    ## no CA atom, e.g. for H_NAG
                    distance[i%N,j%N] = None
                j = j+1
            i = i+1
    else: 
        return []
    return N, distance

We compute the persistent diagram associated to the distance between CA residues.

In [9]:
%%time
# create parser
parser = PDBParser()

#create Rips
rips = Rips()

all_alpha_dgms = {}
all_beta_dgms = {}
alpha_beta_dgms = {}
d = {'all_alphas': all_alpha_dgms, 'all_betas': all_beta_dgms, 'alpha_beta': alpha_beta_dgms}
class_proteins = [all_alphas, all_betas, alpha_beta]
name_class = ['all_alphas', 'all_betas', 'alpha_beta']
all_alpha_atoms = {}
all_beta_atoms = {}
alpha_beta_atoms = {}
N_atoms = {'all_alphas': all_alpha_atoms, 'all_betas': all_beta_atoms, 'alpha_beta': alpha_beta_atoms}

for i in range(len(class_proteins)):
    for protein in class_proteins[i]:
        try: 
            # read structure from file
            structure = parser.get_structure(protein, '../data/' + name_class[i] + '/'+ protein + '.pdb')
            #compute distance matrix among CA atoms
            N, distance = get_distance_matrix(structure)
            if len(distance)>0:
                d[name_class[i]][protein] = rips.fit_transform(distance, distance_matrix=True)
                N_atoms[name_class[i]][protein] = N
        except:
            continue


Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)
Wall time: 2min 17s


### All alpha

In [2]:
#Plot some persistence diagrams associated to proteins all_alpha

### All beta

In [3]:
#Plot some persistence diagrams associated to proteins all_beta

### Alpha & beta


In [4]:
#Plot some persistence diagrams associated to proteins alpha_beta

**Question:** Can you see any salient difference between the diagrams associated to proteins of different classes?

## 2. Machine Learning

The strategy now is to compute a set of descriptors of the most salient features associated to each persistence diagram, to then feed a machine learning classifier. 

### Feature engineering

#### **Description of the features**
- The length of the second longest **Betti 0** bar.
- The length of the third longest **Betti 0** bar.
- The summation of lengths of all **Betti 0** bars except for those exceed the max filtration value.
- The average length of **Betti 0** bars except for those exceed the max filtration value.
- The onset value of the longest **Betti 1** bar.
- The length of the longest **Betti 1** bar.
- The smallest onset value of the **Betti 1** bar that is longer than 1.5Å.
- The average of the middle point values of all the **Betti 1** bars that are longer than 1.5Å.
- The number of **Betti 1** bars that locate at [4.5, 5.5]Å, divided by the number of atoms.
- The number of **Betti 1** bars that locate at [3.5, 4.5)Å and (5.5, 6.5]Å, divided by the number of atoms.
- The summation of lengths of all the **Betti 1** bars except for those exceed the max filtration value.
- The average length of **Betti 1** bars except for those exceed the max filtration value.
- The onset value of the first **Betti 2** bar that ends after a given number.

In [13]:
# The list of the codes of all the proteins
all_proteins = list(d['all_alphas'].keys()) +  list(d['all_betas'].keys()) + list(d['alpha_beta'].keys())

In [14]:
# The list of the labels of all the proteins (its secondary structure)
labels = ['all alpha' for x in range(len(d['all_alphas'].keys()))] + ['all beta' for x in range(len(d['all_betas'].keys()))] + ['alpha beta' for x in range(len(d['alpha_beta'].keys()))]

In [15]:
# We create a dataframe with this information
data = pd.DataFrame({'Protein': all_proteins, 'Second structure': labels})

In [16]:
# We now compute the features associated to each persistence diagram
sec_longest_b0 = []
thi_longest_b0 = []
sum_b0 = []
mean_b0 = []
onset_longest_b1 = []
length_longest_b1 = []
smallest_onset_b1 = []
average_middle_b1 = []
sum_b1 = []
mean_b1 = []
b1_segment = []
b1_out_segment = []

for i in range(len(class_proteins)):
    for protein in d[name_class[i]].keys():
        dgm = d[name_class[i]][protein]
        atoms = N_atoms[name_class[i]][protein]
        if dgm[0][dgm[0][:, 1]<10000].sum()>1:
            sec_longest_b0.append(dgm[0][dgm[0][:, 1]<10000][-2][1])
        else:
            sec_longest_b0.append(0)
        if dgm[0][dgm[0][:, 1]<10000].sum()>2:
            thi_longest_b0.append(dgm[0][dgm[0][:, 1]<10000][-3][1])
        else:
            thi_longest_b0.append(0)
        if dgm[0][dgm[0][:, 1]<10000].sum()>0:
            sum_b0.append(dgm[0][dgm[0][:, 1]<10000].sum())
            mean_b0.append(dgm[0][dgm[0][:, 1]<10000].mean())
        else:
            sum_b0.append(0)
            mean_b0.append(0)
        if len(dgm[1])>1:
            onset_longest_b1.append(dgm[1][-1][0])
            length_longest_b1.append(dgm[1][-1][1]-dgm[1][-1][0])
        else: 
            onset_longest_b1.append(0)
            length_longest_b1.append(0)
        if len(dgm[1])>0:
            smallest_onset_b1.append(dgm[1][:,0].min())
            average_middle_b1.append(((dgm[1][:, 1]-dgm[1][:, 0])/2).mean())
            sum_b1.append((dgm[1][:, 1]-dgm[1][:, 0]).sum())
            mean_b1.append((dgm[1][:, 1]-dgm[1][:, 0]).mean())
            b1_segment.append((((dgm[1][:, 1]<=5.5) & (dgm[1][:, 0]>=4.5)).sum())/atoms)
            b1_out_segment.append((((dgm[1][:, 1]<=6.5) & (dgm[1][:, 0]>5.5)).sum() + (((dgm[1][:, 1]<4.5) & (dgm[1][:, 0]>=3.5)).sum()))/atoms)
        else:
            smallest_onset_b1.append(0)
            average_middle_b1.append(0)
            sum_b1.append(0)
            mean_b1.append(0) 
            b1_segment.append(0)
            b1_out_segment.append(0)

In [ ]:
# We now put all the features in the same dataframe.

features = [sec_longest_b0, thi_longest_b0, sum_b0, mean_b0, onset_longest_b1, length_longest_b1, smallest_onset_b1, average_middle_b1, sum_b1, mean_b1, b1_segment, b1_out_segment] 
feature_names = ['sec_longest_b0', 'thi_longest_b0', 'sum_b0', 'mean_b0', 'onset_longest_b1', 'length_longest_b1', 'smallest_onset_b1', 'average_middle_b1', 'sum_b1', 'mean_b1', 'b1_segment', 'b1_out_segment'] 

for i in range(len(features)):
    data[feature_names[i]] = features[i]

In [ ]:
#This is the dataframe with the code of each protein, is second structure, and the list of its attributes derived from teh persistence diagram.
data.head()

### Machine Learning Classifier

We train a classifier of the secondary structure of proteins from the geometric features extracted from its persistence diagrams. 

In [ ]:
# Choose your favorite (simple) machine learning classifier (such as Random Forest, SVM, others).
# Split the data in train/test set witgh rate 0.8/0.2
# Train your classifier with the train set (to predict the secondary structure)
# Predict the secondary structure of the test set

**Question:** What's the accuracy of your ML classifier? Between which classes is there the highest rate of misclassification? Plot the confusion matrix.